In [2]:
import cv2
import numpy as np

# ---- Vstup / výstup ----
in_video  = "pvi_cv10_video_in.mp4"
out_video = "pvi_cv10_video_out.mp4"
orange_pattern = "pvi_cv10_vzor_pomeranc.bmp"

cap = cv2.VideoCapture(in_video)
if not cap.isOpened():
    raise RuntimeError("Nepodařilo se otevřít vstupní video.")

ret, frame = cap.read()
if not ret:
    raise RuntimeError("Video je prázdné.")

h, w = frame.shape[:2]

fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(out_video, fourcc, 10.0, (w, h))   # 10 snímků/s

# ---- Parametry algoritmů ----
alpha = 0.005          # pomalejší učení pozadí
th_bg = 40             # threshold pro FG masku
min_area = 1500        # min. plocha kontury
cut_threshold = 25     # práh pro "střih"
orange_ratio_thresh = 0.2   # kolik oranžových pixelů v bboxu = „je to pomeranč“

prev_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
bg_model = None
segment_id = 0

# ---- Příprava pro CamShift + barvu pomeranče ----
roi_hist = None
track_window = None
orange_lower = None
orange_upper = None

orange = cv2.imread(orange_pattern)
if orange is not None:
    hsv_orange = cv2.cvtColor(orange, cv2.COLOR_BGR2HSV)

    # 1) histogram pro CamShift (klasika)
    roi_hist = cv2.calcHist([hsv_orange], [0, 1], None,
                            [180, 256], [0, 180, 0, 256])
    cv2.normalize(roi_hist, roi_hist, 0, 255, cv2.NORM_MINMAX)

    # 2) automatický HSV rozsah pomeranče z percentilů
    h_ch, s_ch, v_ch = cv2.split(hsv_orange)
    h_low  = np.percentile(h_ch, 5)
    h_high = np.percentile(h_ch, 95)
    s_low  = np.percentile(s_ch, 5)
    v_low  = np.percentile(v_ch, 5)

    # trošku rozšíříme rozsah a ořízneme na platné hodnoty
    orange_lower = (max(0,   int(h_low  - 5)),
                    max(0,   int(s_low  - 30)),
                    max(0,   int(v_low  - 30)))
    orange_upper = (min(179, int(h_high + 5)),
                    255,
                    255)

term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1)

while True:
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    diff = cv2.absdiff(gray, prev_gray)
    mean_diff = diff.mean()

    # ---- Segmentace videa ----
    if mean_diff > cut_threshold:
        segment_id += 1
        bg_model = None
        track_window = None

    prev_gray = gray.copy()

    # ---- Background Subtraction ----
    if bg_model is None:
        bg_model = gray.astype("float32")
    else:
        cv2.accumulateWeighted(gray, bg_model, alpha)

    bg_uint8 = cv2.convertScaleAbs(bg_model)
    fg = cv2.absdiff(gray, bg_uint8)
    fg = cv2.GaussianBlur(fg, (7, 7), 0)
    _, fg_mask = cv2.threshold(fg, th_bg, 255, cv2.THRESH_BINARY)

    kernel = np.ones((3, 3), np.uint8)
    fg_mask = cv2.morphologyEx(fg_mask, cv2.MORPH_OPEN, kernel, iterations=2)
    fg_mask = cv2.dilate(fg_mask, kernel, iterations=2)

    contours, _ = cv2.findContours(fg_mask, cv2.RETR_EXTERNAL,
                                   cv2.CHAIN_APPROX_SIMPLE)
    big_contours = [c for c in contours if cv2.contourArea(c) >= min_area]

    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # maska oranžové barvy v aktuálním snímku
    orange_mask = None
    if orange_lower is not None:
        orange_mask = cv2.inRange(hsv, orange_lower, orange_upper)

    # backprojection pro CamShift
    backproj = None
    if roi_hist is not None:
        backproj = cv2.calcBackProject([hsv], [0, 1], roi_hist,
                                       [0, 180, 0, 256], 1)

    # ---- 1) pokus najít pomeranč mezi pohybujícími se oblastmi ----
    best_box = None
    if orange_mask is not None:
        best_ratio = 0.0
        for c in big_contours:
            x, y, ww, hh = cv2.boundingRect(c)
            roi_orange = orange_mask[y:y+hh, x:x+ww]
            if roi_orange.size == 0:
                continue
            orange_pixels = cv2.countNonZero(roi_orange)
            ratio = orange_pixels / float(ww * hh)

            if ratio > orange_ratio_thresh and ratio > best_ratio:
                best_ratio = ratio
                best_box = (x, y, ww, hh)

    orange_found = best_box is not None

    # ---- pokud je pomeranč, focus na něj ----
    if orange_found:
        x, y, ww, hh = best_box
        cv2.rectangle(frame, (x, y), (x + ww, y + hh), (0, 255, 0), 2)
        track_window = best_box
    else:
        # jinak detekuj a označ všechno, co se hýbe
        for c in big_contours:
            x, y, ww, hh = cv2.boundingRect(c)
            cv2.rectangle(frame, (x, y), (x + ww, y + hh), (0, 255, 0), 2)

    # ---- CamShift – sleduje pomeranč, pokud existuje track_window ----
    if track_window is not None and backproj is not None:
        ret_cs, track_window = cv2.CamShift(backproj, track_window, term_crit)
        pts = cv2.boxPoints(ret_cs)
        pts = np.int0(pts)
        cv2.polylines(frame, [pts], True, (0, 255, 0), 2)

        # když z okna mizí oranžová barva, zrušíme track
        x, y, ww, hh = track_window
        roi_orange_tr = orange_mask[y:y+hh, x:x+ww] if orange_mask is not None else None
        if roi_orange_tr is None or roi_orange_tr.size == 0 or \
                cv2.countNonZero(roi_orange_tr) / float(ww * hh) < orange_ratio_thresh / 2:
            track_window = None

    cv2.putText(frame, f"seg {segment_id}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 0), 2)

    out.write(frame)

    ret, frame = cap.read()
    if not ret:
        break

cap.release()
out.release()


/tmp/ipykernel_23696/727533224.py:143: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  pts = np.int0(pts)


In [3]:
# ---- pomocná funkce: sloučení všech obdélníků do jednoho ----
def merge_boxes(boxes):
    """
    boxes: list of (x, y, w, h)
    vrátí seznam s nejvýše jedním obdélníkem, který pokrývá všechny vstupní.
    """
    if not boxes:
        return []

    # vezmeme min x,y a max x+w, y+h přes všechny boxy
    min_x = min(x for (x, y, w, h) in boxes)
    min_y = min(y for (x, y, w, h) in boxes)
    max_x = max(x + w for (x, y, w, h) in boxes)
    max_y = max(y + h for (x, y, w, h) in boxes)

    merged_box = (min_x, min_y, max_x - min_x, max_y - min_y)
    return [merged_box]


In [4]:
# ---- Vstup / výstup ----
in_video  = "pvi_cv10_video_in.mp4"
out_video = "pvi_cv10_video_out.mp4"
orange_pattern = "pvi_cv10_vzor_pomeranc.bmp"

cap = cv2.VideoCapture(in_video)
if not cap.isOpened():
    raise RuntimeError("Nepodařilo se otevřít vstupní video.")

ret, frame = cap.read()
if not ret:
    raise RuntimeError("Video je prázdné.")

h, w = frame.shape[:2]

fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(out_video, fourcc, 10.0, (w, h))   # 10 snímků/s

# ---- Parametry algoritmů ----
alpha = 0.005          # pomalejší učení pozadí
th_bg = 40             # threshold pro FG masku
min_area = 1500        # min. plocha kontury
cut_threshold = 25     # práh pro "střih"
orange_ratio_thresh = 0.2   # (už vlastně nepotřebujeme, ale nechávám)

prev_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
bg_model = None
segment_id = 0

# ---- Příprava pro CamShift + barvu pomeranče ----
roi_hist = None
track_window = None
orange_lower = None
orange_upper = None

orange = cv2.imread(orange_pattern)
if orange is not None:
    hsv_orange = cv2.cvtColor(orange, cv2.COLOR_BGR2HSV)

    # 1) histogram pro CamShift (klasika)
    roi_hist = cv2.calcHist([hsv_orange], [0, 1], None,
                            [180, 256], [0, 180, 0, 256])
    cv2.normalize(roi_hist, roi_hist, 0, 255, cv2.NORM_MINMAX)

    # 2) automatický HSV rozsah pomeranče z percentilů
    h_ch, s_ch, v_ch = cv2.split(hsv_orange)
    h_low  = np.percentile(h_ch, 5)
    h_high = np.percentile(h_ch, 95)
    s_low  = np.percentile(s_ch, 5)
    v_low  = np.percentile(v_ch, 5)

    # trošku rozšíříme rozsah a ořízneme na platné hodnoty
    orange_lower = (max(0,   int(h_low  - 5)),
                    max(0,   int(s_low  - 30)),
                    max(0,   int(v_low  - 30)))
    orange_upper = (min(179, int(h_high + 5)),
                    255,
                    255)

term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1)

while True:
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    diff = cv2.absdiff(gray, prev_gray)
    mean_diff = diff.mean()

    # ---- Segmentace videa ----
    if mean_diff > cut_threshold:
        segment_id += 1
        bg_model = None
        track_window = None

    prev_gray = gray.copy()

    # ---- Background Subtraction ----
    if bg_model is None:
        bg_model = gray.astype("float32")
    else:
        cv2.accumulateWeighted(gray, bg_model, alpha)

    bg_uint8 = cv2.convertScaleAbs(bg_model)
    fg = cv2.absdiff(gray, bg_uint8)
    fg = cv2.GaussianBlur(fg, (7, 7), 0)
    _, fg_mask = cv2.threshold(fg, th_bg, 255, cv2.THRESH_BINARY)

    kernel = np.ones((3, 3), np.uint8)
    fg_mask = cv2.morphologyEx(fg_mask, cv2.MORPH_OPEN, kernel, iterations=2)
    fg_mask = cv2.dilate(fg_mask, kernel, iterations=2)

    contours, _ = cv2.findContours(fg_mask, cv2.RETR_EXTERNAL,
                                   cv2.CHAIN_APPROX_SIMPLE)
    big_contours = [c for c in contours if cv2.contourArea(c) >= min_area]

    # převod na obdélníky a sloučení dotýkajících se (pro fallback – „cokoliv se hýbe“)
    motion_boxes = [cv2.boundingRect(c) for c in big_contours]
    motion_boxes = merge_boxes(motion_boxes)

    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # maska oranžové barvy v aktuálním snímku
    orange_mask = None
    if orange_lower is not None:
        orange_mask = cv2.inRange(hsv, orange_lower, orange_upper)

    # backprojection pro CamShift
    backproj = None
    if roi_hist is not None:
        backproj = cv2.calcBackProject([hsv], [0, 1], roi_hist,
                                       [0, 180, 0, 256], 1)

    # ---- 1) POKUS NAJÍT POMERANČ: kontury na (orange_mask ∧ fg_mask) ----
    best_box = None
    if orange_mask is not None:
        # pouze oranžové pixely, které se zároveň hýbou
        orange_moving = cv2.bitwise_and(orange_mask, orange_mask, mask=fg_mask)
        contours_orange, _ = cv2.findContours(orange_moving, cv2.RETR_EXTERNAL,
                                              cv2.CHAIN_APPROX_SIMPLE)
        orange_contours_big = [c for c in contours_orange if cv2.contourArea(c) >= min_area / 5]

        orange_boxes = [cv2.boundingRect(c) for c in orange_contours_big]
        orange_boxes = merge_boxes(orange_boxes)

        if orange_boxes:
            # vybereš si třeba největší oranžový box
            best_box = max(orange_boxes, key=lambda b: b[2] * b[3])

    orange_found = best_box is not None

    # ---- pokud je pomeranč, focus na něj ----
    if orange_found:
        x, y, ww, hh = best_box
        cv2.rectangle(frame, (x, y), (x + ww, y + hh), (0, 255, 0), 2)
        track_window = best_box
    else:
        # jinak detekuj a označ všechno, co se hýbe (sloučené motion_boxes)
        for (x, y, ww, hh) in motion_boxes:
            cv2.rectangle(frame, (x, y), (x + ww, y + hh), (0, 255, 0), 2)

    # ---- CamShift – sleduje pomeranč, pokud existuje track_window ----
    if track_window is not None and backproj is not None:
        ret_cs, track_window = cv2.CamShift(backproj, track_window, term_crit)

        # NEKRESLÍME rotovaný box, jen případně používáme track_window interně
        x, y, ww, hh = track_window
        # (pokud chceš, můžeš kreslit i tento AABB, ale už máš best_box nahoře)

        # když z okna mizí oranžová barva, zrušíme track
        roi_orange_tr = orange_mask[y:y+hh, x:x+ww] if orange_mask is not None else None
        if roi_orange_tr is None or roi_orange_tr.size == 0 or \
                cv2.countNonZero(roi_orange_tr) / float(ww * hh) < orange_ratio_thresh / 2:
            track_window = None

    cv2.putText(frame, f"seg {segment_id}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 0), 2)

    out.write(frame)

    ret, frame = cap.read()
    if not ret:
        break

cap.release()
out.release()



In [1]:
import cv2
import numpy as np

# ---- pomocná funkce: sloučení dotýkajících se / překrývajících se obdélníků ----
def merge_boxes(boxes):
    """
    boxes: list of (x, y, w, h)
    vrátí nový seznam, kde všechny dotýkající se/překrývající se obdélníky
    jsou sloučeny do jednoho (union).
    """
    if not boxes:
        return []

    boxes = boxes.copy()
    changed = True

    while changed:
        changed = False
        merged = []
        used = [False] * len(boxes)

        for i in range(len(boxes)):
            if used[i]:
                continue

            x1, y1, w1, h1 = boxes[i]
            ax1, ay1 = x1, y1
            ax2, ay2 = x1 + w1, y1 + h1

            for j in range(i + 1, len(boxes)):
                if used[j]:
                    continue

                x2, y2, w2, h2 = boxes[j]
                bx1, by1 = x2, y2
                bx2, by2 = x2 + w2, y2 + h2

                # dotyk nebo překryv (hranice se aspoň dotknou)
                if not (ax2 < bx1 or bx2 < ax1 or ay2 < by1 or by2 < ay1):
                    ax1 = min(ax1, bx1)
                    ay1 = min(ay1, by1)
                    ax2 = max(ax2, bx2)
                    ay2 = max(ay2, by2)
                    used[j] = True
                    changed = True

            used[i] = True
            merged.append((ax1, ay1, ax2 - ax1, ay2 - ay1))

        boxes = merged

    return boxes


# ---- Vstup / výstup ----
in_video  = "pvi_cv10_video_in.mp4"
out_video = "pvi_cv10_video_out.mp4"
orange_pattern = "pvi_cv10_vzor_pomeranc.bmp"

cap = cv2.VideoCapture(in_video)
if not cap.isOpened():
    raise RuntimeError("Nepodařilo se otevřít vstupní video.")

ret, frame = cap.read()
if not ret:
    raise RuntimeError("Video je prázdné.")

h, w = frame.shape[:2]

fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(out_video, fourcc, 10.0, (w, h))   # 10 snímků/s

# ---- Parametry algoritmů ----
alpha = 0.005          # pomalejší učení pozadí
th_bg = 40             # threshold pro FG masku
min_area = 1500        # min. plocha kontury
cut_threshold = 25     # práh pro "střih"
orange_ratio_thresh = 0.2   # (používá se jen pro kontrolu oranžové)

prev_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
bg_model = None
segment_id = 0

# ---- Příprava pro oranžový vzor ----
roi_hist = None
orange_lower = None
orange_upper = None

orange = cv2.imread(orange_pattern)
if orange is not None:
    hsv_orange = cv2.cvtColor(orange, cv2.COLOR_BGR2HSV)

    # histogram pomeranče (backprojection můžeš použít pro ladění)
    roi_hist = cv2.calcHist([hsv_orange], [0, 1], None,
                            [180, 256], [0, 180, 0, 256])
    cv2.normalize(roi_hist, roi_hist, 0, 255, cv2.NORM_MINMAX)

    # automatický HSV rozsah pomeranče z percentilů
    h_ch, s_ch, v_ch = cv2.split(hsv_orange)
    h_low  = np.percentile(h_ch, 5)
    h_high = np.percentile(h_ch, 95)
    s_low  = np.percentile(s_ch, 5)
    v_low  = np.percentile(v_ch, 5)

    orange_lower = (max(0,   int(h_low  - 5)),
                    max(0,   int(s_low  - 30)),
                    max(0,   int(v_low  - 30)))
    orange_upper = (min(179, int(h_high + 5)),
                    255,
                    255)

# ---- CamShift nastavení pro "jiný objekt" (ne pomeranč) ----
term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1)
other_track_window = None   # bbox pro jiný objekt
other_hist = None           # histogram jiného objektu (fixní, bez adaptace)

while True:
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    diff = cv2.absdiff(gray, prev_gray)
    mean_diff = diff.mean()

    # ---- Segmentace videa ----
    if mean_diff > cut_threshold:
        segment_id += 1
        bg_model = None
        other_track_window = None
        other_hist = None

    prev_gray = gray.copy()

    # ---- Background Subtraction ----
    if bg_model is None:
        bg_model = gray.astype("float32")
    else:
        cv2.accumulateWeighted(gray, bg_model, alpha)

    bg_uint8 = cv2.convertScaleAbs(bg_model)
    fg = cv2.absdiff(gray, bg_uint8)
    fg = cv2.GaussianBlur(fg, (7, 7), 0)
    _, fg_mask = cv2.threshold(fg, th_bg, 255, cv2.THRESH_BINARY)

    kernel = np.ones((3, 3), np.uint8)
    fg_mask = cv2.morphologyEx(fg_mask, cv2.MORPH_OPEN, kernel, iterations=2)
    fg_mask = cv2.dilate(fg_mask, kernel, iterations=2)

    contours, _ = cv2.findContours(fg_mask, cv2.RETR_EXTERNAL,
                                   cv2.CHAIN_APPROX_SIMPLE)
    big_contours = [c for c in contours if cv2.contourArea(c) >= min_area]

    # fallback: cokoliv se hýbe (sloučené do jednoho boxu)
    motion_boxes = [cv2.boundingRect(c) for c in big_contours]
    motion_boxes = merge_boxes(motion_boxes)

    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # maska oranžové barvy v aktuálním snímku
    orange_mask = None
    if orange_lower is not None:
        orange_mask = cv2.inRange(hsv, orange_lower, orange_upper)

    # backprojection pomeranče (jen kdybys chtěl pro debug)
    backproj_orange = None
    if roi_hist is not None:
        backproj_orange = cv2.calcBackProject([hsv], [0, 1], roi_hist,
                                              [0, 180, 0, 256], 1)

    # ---- Najít pomeranč: kontury na (orange_mask ∧ fg_mask) ----
    best_box = None
    if orange_mask is not None:
        orange_moving = cv2.bitwise_and(orange_mask, orange_mask, mask=fg_mask)
        contours_orange, _ = cv2.findContours(orange_moving, cv2.RETR_EXTERNAL,
                                              cv2.CHAIN_APPROX_SIMPLE)
        orange_contours_big = [c for c in contours_orange
                               if cv2.contourArea(c) >= min_area / 5]

        orange_boxes = [cv2.boundingRect(c) for c in orange_contours_big]
        orange_boxes = merge_boxes(orange_boxes)

        if orange_boxes:
            best_box = max(orange_boxes, key=lambda b: b[2] * b[3])

    orange_found = best_box is not None

    # ============================================================
    # 1) Pokud je pomeranč → zobraz ho a vypni CamShift na jiný objekt
    # ============================================================
    if orange_found:
        x, y, ww, hh = best_box
        cv2.rectangle(frame, (x, y), (x + ww, y + hh), (0, 255, 0), 2)

        # když sledujeme pomeranč, nechceme sledovat nic jiného
        other_track_window = None
        other_hist = None

    # ============================================================
    # 2) Jinak použij FIXNÍ CamShift na jiný objekt
    # ============================================================
    else:
        # a) inicializace CamShiftu z motion detekce (jen jednou)
        if other_track_window is None and motion_boxes:
            # vememe největší pohybující se box
            x, y, ww, hh = max(motion_boxes, key=lambda b: b[2] * b[3])
            other_track_window = (x, y, ww, hh)

            # fixní histogram objektu – spočítá se JEDNOU
            roi = hsv[y:y+hh, x:x+ww]
            mask_roi = fg_mask[y:y+hh, x:x+ww]   # jen FG část
            other_hist = cv2.calcHist([roi], [0, 1], mask_roi,
                                      [180, 256], [0, 180, 0, 256])
            cv2.normalize(other_hist, other_hist, 0, 255, cv2.NORM_MINMAX)

            # nakreslíme inicializační box
            cv2.rectangle(frame, (x, y), (x + ww, y + hh), (0, 255, 0), 2)

        # b) pokud CamShift už běží → sleduj objekt podle fixního histogramu
        elif other_track_window is not None and other_hist is not None:
            backproj_other = cv2.calcBackProject([hsv], [0, 1], other_hist,
                                                 [0, 180, 0, 256], 1)

            ret_cs, other_track_window = cv2.CamShift(backproj_other,
                                                      other_track_window,
                                                      term_crit)

            # použijeme axis-aligned verzi okna (žádný rotovaný polygon)
            x, y, ww, hh = other_track_window
            cv2.rectangle(frame, (x, y), (x + ww, y + hh), (0, 255, 0), 2)

            # jednoduchá podmínka „ztratil jsem objekt?“ – moc malé okno
            if ww * hh < min_area / 2:
                other_track_window = None
                other_hist = None

        # c) fallback, kdyby nebyl ani CamShift ani motion_boxes
        else:
            for (x, y, ww, hh) in motion_boxes:
                cv2.rectangle(frame, (x, y), (x + ww, y + hh), (0, 255, 0), 2)

    cv2.putText(frame, f"seg {segment_id}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 0), 2)

    out.write(frame)

    ret, frame = cap.read()
    if not ret:
        break

cap.release()
out.release()


In [10]:
import cv2
import numpy as np

def merge_boxes(boxes):
    if not boxes:
        return []
    boxes = boxes.copy()
    changed = True
    while changed:
        changed = False
        merged = []
        used = [False] * len(boxes)
        for i in range(len(boxes)):
            if used[i]:
                continue
            x1, y1, w1, h1 = boxes[i]
            ax1, ay1 = x1, y1
            ax2, ay2 = x1 + w1, y1 + h1
            for j in range(i + 1, len(boxes)):
                if used[j]:
                    continue
                x2, y2, w2, h2 = boxes[j]
                bx1, by1 = x2, y2
                bx2, by2 = x2 + w2, y2 + h2
                if not (ax2 < bx1 or bx2 < ax1 or ay2 < by1 or by2 < ay1):
                    ax1 = min(ax1, bx1)
                    ay1 = min(ay1, by1)
                    ax2 = max(ax2, bx2)
                    ay2 = max(ay2, by2)
                    used[j] = True
                    changed = True
            used[i] = True
            merged.append((ax1, ay1, ax2 - ax1, ay2 - ay1))
        boxes = merged
    return boxes


# ---- Vstup / výstup ----
in_video  = "pvi_cv10_video_in.mp4"
out_video = "pvi_cv10_video_out.mp4"
orange_pattern = "pvi_cv10_vzor_pomeranc.bmp"

cap = cv2.VideoCapture(in_video)
if not cap.isOpened():
    raise RuntimeError("Nepodařilo se otevřít vstupní video.")

ret, frame = cap.read()
if not ret:
    raise RuntimeError("Video je prázdné.")

h, w = frame.shape[:2]
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(out_video, fourcc, 10.0, (w, h))

# ---- Parametry algoritmů ----
alpha = 0.005
th_bg = 40
min_area = 1500
cut_threshold = 25

prev_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
bg_model = None
segment_id = 0

# ---- Oranžový vzor (HSV rozsah) ----
orange = cv2.imread(orange_pattern)
orange_lower = None
orange_upper = None
if orange is not None:
    hsv_orange = cv2.cvtColor(orange, cv2.COLOR_BGR2HSV)
    h_ch, s_ch, v_ch = cv2.split(hsv_orange)

    h_low  = np.percentile(h_ch, 10)
    h_high = np.percentile(h_ch, 90)
    s_low  = np.percentile(s_ch, 20)
    v_low  = np.percentile(v_ch, 20)

    s_min = max(100, int(s_low))
    v_min = max(80,  int(v_low))

    orange_lower = (max(0,   int(h_low  - 3)),
                    s_min,
                    v_min)
    orange_upper = (min(179, int(h_high + 3)),
                    255,
                    255)

global_box = None   # (x, y, w, h)

while True:
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    diff = cv2.absdiff(gray, prev_gray)
    mean_diff = diff.mean()

    # segmentace videa
    if mean_diff > cut_threshold:
        segment_id += 1
        bg_model = None
        global_box = None

    prev_gray = gray.copy()

    # background subtraction
    if bg_model is None:
        bg_model = gray.astype("float32")
    else:
        cv2.accumulateWeighted(gray, bg_model, alpha)

    bg_uint8 = cv2.convertScaleAbs(bg_model)
    fg = cv2.absdiff(gray, bg_uint8)
    fg = cv2.GaussianBlur(fg, (7, 7), 0)
    _, fg_mask = cv2.threshold(fg, th_bg, 255, cv2.THRESH_BINARY)

    kernel = np.ones((3, 3), np.uint8)
    fg_mask = cv2.morphologyEx(fg_mask, cv2.MORPH_OPEN, kernel, iterations=2)
    fg_mask = cv2.dilate(fg_mask, kernel, iterations=2)

    contours, _ = cv2.findContours(fg_mask, cv2.RETR_EXTERNAL,
                                   cv2.CHAIN_APPROX_SIMPLE)
    big_contours = [c for c in contours if cv2.contourArea(c) >= min_area]
    motion_boxes = merge_boxes([cv2.boundingRect(c) for c in big_contours])

    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # ---- ORANŽOVÁ: jen v seg1, s filtrem na "jak moc oranžová" ----
    best_orange_box = None
    if orange_lower is not None and segment_id == 1:
        orange_mask = cv2.inRange(hsv, orange_lower, orange_upper)

        orange_mask = cv2.morphologyEx(orange_mask, cv2.MORPH_OPEN, kernel, iterations=1)
        orange_mask = cv2.dilate(orange_mask, kernel, iterations=1)

        contours_orange, _ = cv2.findContours(orange_mask, cv2.RETR_EXTERNAL,
                                              cv2.CHAIN_APPROX_SIMPLE)

        orange_boxes = []
        for c in contours_orange:
            area = cv2.contourArea(c)
            if area < min_area / 10:
                continue

            x, y, ww, hh = cv2.boundingRect(c)
            roi = orange_mask[y:y+hh, x:x+ww]
            if roi.size == 0:
                continue

            orange_pixels = cv2.countNonZero(roi)
            ratio = orange_pixels / float(ww * hh)
            if ratio < 0.4:   # většina boxu musí být fakt oranžová
                continue

            orange_boxes.append((x, y, ww, hh))

        orange_boxes = merge_boxes(orange_boxes)
        if orange_boxes:
            best_orange_box = max(orange_boxes, key=lambda b: b[2] * b[3])

    # výběr kandidáta pro tento frame
    if segment_id == 1:
        candidate = best_orange_box        # seg1: jen pomeranč
    else:
        candidate = max(motion_boxes, key=lambda b: b[2] * b[3]) if motion_boxes else None

    # ---- UPDATE GLOBAL_BOX ----
    if candidate is not None:
        cx, cy, cw, ch = candidate
        if global_box is None:
            # první dobrý box → zamkneme velikost
            if segment_id == 1:
                global_box = (cx, cy, cw, ch)
            else:
                global_box = candidate
        else:
            gx, gy, gw, gh = global_box
            if segment_id == 1:
                # SEG1: držíme PEPEVNOU velikost, měníme jen pozici
                # zarovnáme vlevo nahoře podle aktuální detekce
                new_x = cx
                new_y = cy

                # případné ořezání na hranice obrazu
                new_x = max(0, min(new_x, w - gw))
                new_y = max(0, min(new_y, h - gh))

                global_box = (new_x, new_y, gw, gh)
            else:
                # mimo seg1 trochu bráníme zmenšení (můžeš si upravit nebo vypnout)
                g_area = gw * gh
                c_area = cw * ch
                if c_area >= 0.5 * g_area:
                    global_box = candidate

    # kreslení
    if global_box is not None:
        x, y, ww, hh = global_box
        cv2.rectangle(frame, (x, y), (x + ww, y + hh), (0, 255, 0), 2)

    cv2.putText(frame, f"seg {segment_id}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 0), 2)

    out.write(frame)

    ret, frame = cap.read()
    if not ret:
        break

cap.release()
out.release()
